# worldometers table scraping

## extracting text form html table

In [1]:
import requests
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
from pprint import pprint

fileDay = datetime.date.today().strftime('%m%d')

df = None

colList = []  
country = []
totCase = []
newCase = []
totDeath = []
newDeath = []
totRecov = []
actCase = []
serious = []
totCasePer1M = []

allList = [colList, country, totCase, newCase, totDeath, newDeath, totRecov, actCase, serious, totCasePer1M]

def colClear():
    for i in allList:
        i.clear()

def scrapingWorldometers(yda=False):
    url = "https://www.worldometers.info/coronavirus/"
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    
    if yda:
        table = soup.select_one('table#main_table_countries_yesterday')
    else:
        table = soup.select_one('table#main_table_countries_today')
    #pprint(table)

    columns = table.select('tr')
    isCol = True
    colClear()

    for col in columns:
        if isCol:
            cols = col.select('th')
            for c in cols:
                colList.append(c.text)
            isCol = False
        else:
            rows = col.select('td')
            country.append(rows[0].text)
            totCase.append(rows[1].text)
            newCase.append(rows[2].text)
            totDeath.append(rows[3].text)
            newDeath.append(rows[4].text)
            totRecov.append(rows[5].text)
            actCase.append(rows[6].text)
            serious.append(rows[7].text)
            totCasePer1M.append(rows[8].text)
    # 'Tot\xa0Cases/1M pop' => 'Tot Cases/1M pop'
    colList[8] = 'Tot Cases/1M pop'

## making dataframe, json, csv

In [2]:
def dfProcessing():
    
    # Réunion, Curaçao => Reunion, Curacao 
    idx = country.index("Réunion")
    country[idx] = "Reunion"
    
    idx = country.index("Curaçao")
    country[idx] = "Curacao"
    
    df = pd.DataFrame(columns=colList)
    df.iloc[:,0] = country
    df.iloc[:,1] = totCase
    df.iloc[:,2] = newCase
    df.iloc[:,3] = totDeath
    df.iloc[:,4] = newDeath
    df.iloc[:,5] = totRecov
    df.iloc[:,6] = actCase
    df.iloc[:,7] = serious
    df.iloc[:,8] = totCasePer1M
    
    df = df.set_index('Country,Other')

    astypeDict = dict.fromkeys(df.columns, np.int)
    astypeDict['Tot Cases/1M pop'] = np.float

    df = df.replace(',','', regex=True)
    df = df.replace('', 0)
    df = df.replace(' ', 0)
    df.astype(astypeDict).dtypes
    return df

def toCSV(df, yda=False):
    if yda:
        csvFile = './covidDataset/' + fileDay + 'ydaCoronaWorld.csv'
        df.to_csv(csvFile)
        print(fileDay, "yesterday csv complete")
    else:
        csvFile = './covidDataset/' + fileDay + 'tdaCoronaWorld.csv'
        df.to_csv(csvFile)
        print(fileDay, "today csv complete")
    
def toJSON(df, yda=False):
    if yda:
        jsonFile = './covidDataset/' + fileDay + 'ydaCoronaWorld.json'
        df.to_json(jsonFile, orient='table')
        print(fileDay, "yesterday json complete")
    else:
        jsonFile = './covidDataset/' + fileDay + 'tdaCoronaWorld.json'
        df.to_json(jsonFile, orient='table')
        print(fileDay, "today json complete")

## today files

In [3]:
scrapingWorldometers()
df = dfProcessing()
toCSV(df)
toJSON(df)

0320 today csv complete
0320 today json complete


## yesterday files

In [4]:
scrapingWorldometers(yda=True)
df = dfProcessing()
toCSV(df, yda=True)
toJSON(df, yda=True)

0320 yesterday csv complete
0320 yesterday json complete
